# 🚗 Vehicle Search Testing

Quick notebook to test fuzzy matching for vehicle marks and models.


In [1]:
# Install dependencies (run once if needed)
# !pip install rapidfuzz pandas pyarrow


In [2]:
import pandas as pd
from matcher import VehicleMatcher

# Load your data
matcher = VehicleMatcher(parquet_path='vehicles_db.parquet')

print(f"Loaded {len(matcher.df)} vehicles")
print(f"Unique brands: {len(matcher.unique_marks)}")
print(f"\nSample data:")
matcher.df[[matcher.mark_column, matcher.model_column]].head(10)


Loaded 1976 vehicles
Unique brands: 172

Sample data:


,brand,model
0,Toyota,Vitz
1,LADA (ВАЗ),2107
2,Daewoo,Nubira
3,Chevrolet,Aveo
4,Chevrolet,Lacetti
5,Kia,Cerato
6,Kia,Sportage
7,Nissan,Wingroad
8,LADA (ВАЗ),2115
9,Volkswagen,Transporter


## 🔍 Test Search

Enter your test queries below:


In [11]:
# === EDIT THESE VALUES TO TEST ===
query_mark = "лада"   # Brand/mark query (can have typos)
query_model = "веста gfk11"   # Model query (can have typos)
top_k = 5                  # Number of results

# Search
results = matcher.match(query_mark, query_model, top_k=top_k)

print(f"Query: '{query_mark}' + '{query_model}'")
print(f"Found {len(results)} results:\n")

for i, r in enumerate(results, 1):
    print(f"{i}. {r.mark} {r.model}")
    print(f"   Score: {r.combined_score:.1f} (mark: {r.mark_score:.1f}, model: {r.model_score:.1f})")


Query: 'лада' + 'веста gfk11'
Found 5 results:

1. LADA (ВАЗ) Vesta
   Score: 90.0 (mark: 90.0, model: 90.0)
2. LADA (ВАЗ) 2115
   Score: 72.0 (mark: 90.0, model: 60.0)
3. LADA (ВАЗ) 2114
   Score: 72.0 (mark: 90.0, model: 60.0)
4. LADA (ВАЗ) 2110
   Score: 72.0 (mark: 90.0, model: 60.0)
5. LADA (ВАЗ) 2101
   Score: 72.0 (mark: 90.0, model: 60.0)


## 🧪 Batch Testing

Test multiple queries at once:


In [4]:
# Test queries - add your own!
test_queries = [
    ("мерседес", "е класс"),      # Russian
    ("mersedes", "e class"),       # Transliteration with typo
    ("тоета", "камри"),            # Toyota Camry (typo)
    ("бмв", "x5"),                 # BMW X5
    ("volkswagn", "passat"),       # Typo
    ("хёндай", "солярис"),         # Hyundai Solaris
    ("ауди", "а4"),                # Audi A4
    ("форд", "фокус"),             # Ford Focus
]

print("=" * 60)
for query_mark, query_model in test_queries:
    results = matcher.match(query_mark, query_model, top_k=3)
    
    print(f"\n🔍 Query: '{query_mark}' + '{query_model}'")
    if results:
        for r in results:
            print(f"   → {r.mark} {r.model} (score: {r.combined_score:.1f})")
    else:
        print("   ❌ No matches found")
print("\n" + "=" * 60)



🔍 Query: 'мерседес' + 'е класс'
   → Mercedes-Benz Maybach S-Класс (score: 81.0)
   → Mercedes-Benz M-Класс (score: 78.9)
   → Mercedes-Benz B-Класс (score: 78.9)

🔍 Query: 'mersedes' + 'e class'
   → Mercedes-Benz CLA (score: 85.5)
   → Mercedes-Benz CLS (score: 72.5)
   → Mercedes-Benz GLA (score: 67.5)

🔍 Query: 'тоета' + 'камри'
   → Toyota Cami (score: 80.0)
   → Toyota Sprinter Carib (score: 76.0)
   → Toyota Carina E (score: 76.0)

🔍 Query: 'бмв' + 'x5'
   → BMW X5 (score: 100.0)
   → BMW X5 M (score: 94.0)
   → BMW 5 серии (score: 76.0)

🔍 Query: 'volkswagn' + 'passat'
   → Volkswagen Passat (score: 97.9)
   → Volkswagen Passat CC (score: 91.9)
   → Volkswagen Passat (North America) (score: 91.9)

🔍 Query: 'хёндай' + 'солярис'
   → Hyundai Solaris (score: 96.0)
   → Hyundai Genesis Coupe (score: 68.0)
   → Hyundai Pony (score: 67.0)

🔍 Query: 'ауди' + 'а4'
   → Audi A4 (score: 100.0)
   → Audi A4 allroad (score: 94.0)
   → Audi A6 allroad (score: 76.0)

🔍 Query: 'форд' + 'фоку

## 🏷️ Search Brands Only


In [5]:
# Search for similar brands
query = "фолькваген"  # Volkswagen with typo

marks = matcher.find_similar_marks(query, top_k=10)

print(f"Similar brands to '{query}':")
for mark, score in marks:
    print(f"  {mark}: {score:.1f}")


Similar brands to 'фолькваген':
  volkswagen: 73.7
  fso: 72.0
  kia: 60.0
  ford: 60.0
  luxgen: 60.0
  geo: 60.0
  eagle: 60.0
  delage: 60.0
  volvo: 54.0
  opel: 51.4


## 🔧 Interactive Search Helper


In [6]:
def search(mark: str, model: str, top_k: int = 5):
    """Quick search function - returns a nice DataFrame."""
    results = matcher.match(mark, model, top_k=top_k)
    
    data = []
    for r in results:
        data.append({
            'Brand': r.mark,
            'Model': r.model,
            'Score': f"{r.combined_score:.1f}",
            'Mark Score': f"{r.mark_score:.1f}",
            'Model Score': f"{r.model_score:.1f}"
        })
    
    return pd.DataFrame(data)

# Use like this:
search("мерседес", "с класс")


,Brand,Model,Score,Mark Score,Model Score
0,Mercedes-Benz,Maybach S-Класс,81.0,90.0,75.0
1,Mercedes-Benz,A-Класс AMG,80.0,90.0,73.3
2,Mercedes-Benz,M-Класс,78.9,90.0,71.4
3,Mercedes-Benz,B-Класс,78.9,90.0,71.4
4,Mercedes-Benz,C-Класс,78.9,90.0,71.4


## 📊 View All Unique Brands in Database


In [7]:
# Show all unique brands in the database
print(f"All {len(matcher.unique_marks)} unique brands:\n")
for i, mark in enumerate(sorted(matcher.unique_marks), 1):
    print(f"{i:3}. {mark}")


All 172 unique brands:

  1. 
  2. ac
  3. acura
  4. adler
  5. alfa romeo
  6. alpina
  7. amc
  8. ariel
  9. aro
 10. asia
 11. aston martin
 12. auburn
 13. audi
 14. austin
 15. baic
 16. bentley
 17. bmw
 18. borgward
 19. brabus
 20. brilliance
 21. bugatti
 22. buick
 23. byd
 24. cadillac
 25. chana
 26. changan
 27. changfeng
 28. changhe
 29. chery
 30. cheryexeed
 31. chevrolet
 32. chrysler
 33. citroen
 34. dacia
 35. dadi
 36. daewoo
 37. daihatsu
 38. daimler
 39. datsun
 40. delage
 41. derways
 42. dkw
 43. dodge
 44. dongfeng
 45. doninvest
 46. ds
 47. dw hower
 48. eagle
 49. excalibur
 50. faw
 51. ferrari
 52. fiat
 53. fisker
 54. ford
 55. foton
 56. fso
 57. gac
 58. geely
 59. genesis
 60. geo
 61. gmc
 62. great wall
 63. hafei
 64. haima
 65. hanomag
 66. haval
 67. hawtai
 68. heinkel
 69. honda
 70. huanghai
 71. hudson
 72. hummer
 73. hyundai
 74. infiniti
 75. iran khodro
 76. isuzu
 77. iveco
 78. jac
 79. jaguar
 80. jeep
 81. jinbei
 82. jmc
 83. k